# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,23.42,92,100,0.45,CA,1722792949
1,1,bomdila,27.2500,92.4000,11.82,99,100,0.98,IN,1722792953
2,2,polyarnyy,69.1989,33.4478,11.89,73,2,3.31,RU,1722792954
3,3,margaret river,-33.9500,115.0667,14.42,74,92,6.53,AU,1722792956
4,4,tayu,-6.5397,111.0518,24.01,82,41,3.80,ID,1722792957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 580,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_Citydeal_city = city_data_df.loc[(city_data_df["Max Temp"] < 34) & (city_data_df["Max Temp"] > 19) & (city_data_df["Cloudiness"] < 75) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 65)]


# Drop any rows with null values
Ideal_City = Ideal_City.dropna()

# Display sample data
Ideal_City.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,newman,37.3138,-121.0208,30.13,20,0,1.90,US,1722792964
47,47,trancas,-26.2314,-65.2803,22.85,25,0,1.92,AR,1722793006
56,56,qala i naw,34.9874,63.1289,29.35,15,0,2.15,AF,1722793015
57,57,baneh,35.9975,45.8853,22.39,27,1,1.72,IR,1722793016
61,61,zhosaly,45.4878,64.0781,29.76,21,0,3.50,KZ,1722793021
98,98,aytos,42.7000,27.2500,26.69,35,30,1.83,BG,1722793065
133,133,barro alto,-11.7608,-41.9117,27.96,33,9,6.99,BR,1722793113
142,142,nyachera,-0.9000,30.4167,22.82,36,10,0.89,UG,1722793127
144,144,calama,-22.4667,-68.9333,20.27,14,0,9.77,CL,1722793009
145,145,san patricio,28.0170,-97.5169,33.28,42,4,2.01,US,1722793130


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_City[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,newman,US,37.3138,-121.0208,20,
47,trancas,AR,-26.2314,-65.2803,25,
56,qala i naw,AF,34.9874,63.1289,15,
57,baneh,IR,35.9975,45.8853,27,
61,zhosaly,KZ,45.4878,64.0781,21,
98,aytos,BG,42.7000,27.2500,35,
133,barro alto,BR,-11.7608,-41.9117,33,
142,nyachera,UG,-0.9000,30.4167,36,
144,calama,CL,-22.4667,-68.9333,14,
145,san patricio,US,28.0170,-97.5169,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accomodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
trancas - nearest hotel: No hotel found
qala i naw - nearest hotel: No hotel found
baneh - nearest hotel: No hotel found
zhosaly - nearest hotel: No hotel found
aytos - nearest hotel: No hotel found
barro alto - nearest hotel: No hotel found
nyachera - nearest hotel: No hotel found
calama - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
el bayadh - nearest hotel: No hotel found
childress - nearest hotel: No hotel found
isla vista - nearest hotel: No hotel found
oltinko'l - nearest hotel: No hotel found
trindade - nearest hotel: No hotel found
calca - nearest hotel: No hotel found
pueblo west - nearest hotel: No hotel found
kamina - nearest hotel: No hotel found
el idrissia - nearest hotel: No hotel found
la sarre - nearest hotel: No hotel found
turkestan - nearest hotel: No hotel found
al bawiti - nearest hotel: No hotel found
dahab - nearest hotel: No hotel found
happy valley-goose bay - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
11,newman,US,37.3138,-121.0208,20,No hotel found
47,trancas,AR,-26.2314,-65.2803,25,No hotel found
56,qala i naw,AF,34.9874,63.1289,15,No hotel found
57,baneh,IR,35.9975,45.8853,27,No hotel found
61,zhosaly,KZ,45.4878,64.0781,21,No hotel found
98,aytos,BG,42.7000,27.2500,35,No hotel found
133,barro alto,BR,-11.7608,-41.9117,33,No hotel found
142,nyachera,UG,-0.9000,30.4167,36,No hotel found
144,calama,CL,-22.4667,-68.9333,14,No hotel found
145,san patricio,US,28.0170,-97.5169,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lat","Lng","City","Humidity","Hotel Name","Country"] 
)


# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)